<a href="https://colab.research.google.com/github/luigibrb/Ebay-scraper-2021/blob/main/Ebay_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import bs4
import re
import datetime
import pandas as pd
import time
import re

In [ ]:
def scrape_advertisement(url):

  p = requests.get(url)
  page = bs4.BeautifulSoup(p.content,'lxml')
  when = datetime.datetime.now()

  try:
    title = page.h1.text.replace("Details about  \xa0","")
  except:
    title = 'Error: couldnt find the title'

  try:
    description = re.sub("\s\s+" , " ", str(page.find(class_='itemAttr').find(class_='section').table.text.replace('\n',' ').replace('\a',' ')))[0:199]
  except:
    description = 'Error: couldnt find any description'

  if page.find(id='prcIsum_bidPrice') is not None:
    bidprice = float(page.find(id='prcIsum_bidPrice').text.split()[1].replace("$","").replace("'",""))
  else:
    bidprice = 0

  try:
    bidnro = int(page.find(id='qty-test').text)
  except:
    bidnro = 'Error: couldnt find any number of bids'

  try:
    seller = page.find(class_='mbg-nw').text
  except:
    seller = 'Error: couldnt find the seller'

  try:
    sellscore = int(page.find(class_='mbg-l').text.split()[0].replace("(",""))
  except:
    sellscore = 'Error: couldnt find the sellers score'

  try:
    id = page.find(id='descItemNumber').text
  except:
    id = 'Error: couldnt find the id'
    
  return (id, title, bidprice, bidnro, when, seller, sellscore, description)



def scrape_from_ebay(keyword, maximum_number_results = 100):

  actual_number_results = 0
  url = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw='+keyword+'&LH_Auction=1'
  p = requests.get(url)
  page = bs4.BeautifulSoup(p.content)

  page_count = []

  for a in page.find(class_='pagination__items').findChildren("li" , recursive=False):
    page_count.append(a.text)

  output = []

  for page in page_count:
    url = 'https://www.ebay.com/sch/i.html?_from=R40&_nkw='+keyword+'&LH_Auction=1&_pgn='+str(page)
    print("Fetching links from page {} at address {} at this time {}".format(page, url, datetime.datetime.now()))
    p = requests.get(url)
    page = bs4.BeautifulSoup(p.content)

    for ad in page.find('ul', class_='srp-results srp-list clearfix').find_all(class_='s-item__link', href=True):
      print('Scraping data from: {}'.format(ad['href']))
      link_scraped = scrape_advertisement(ad['href'])
      output.append(link_scraped)

      actual_number_results += 1


      if actual_number_results == maximum_number_results:
        return pd.DataFrame(output, columns=['id', 'title', 'bidprice', 'bidnro', 'when', 'seller', 'sellscore', 'description'])
        
      time.sleep(0.1)

  return pd.DataFrame(output, columns=['id', 'title', 'bidprice', 'bidnro', 'when', 'seller', 'sellscore', 'description'])
